# 결재 댓글 검증 및 추출

## 목적
1. **1단계**: DB에 있는 100개 댓글 기준으로 HTML 파싱 검증
2. **2단계**: 전체 HTML 스캔 및 누락 댓글 추출 (1단계 100% 일치 시)
3. **3단계**: DB INSERT

---
## 셀 1: 설정 및 라이브러리

In [ ]:
# 필요한 라이브러리 설치 (최초 1회)
# !pip install beautifulsoup4 lxml mysql-connector-python pandas tqdm

In [1]:
import os
import re
import glob
from datetime import datetime
from bs4 import BeautifulSoup, NavigableString
import mysql.connector
import pandas as pd
from tqdm import tqdm
from collections import defaultdict

# 경고 무시
import warnings
warnings.filterwarnings('ignore')

print("라이브러리 로드 완료")

라이브러리 로드 완료


In [2]:
# ========== 설정 ==========

# HTML 파일 폴더 경로 (4개)
HTML_FOLDERS = [
    r"C:\Users\LEEJUHWAN\Downloads\2010-01-01~2010-12-31\html\결재",
    r"C:\Users\LEEJUHWAN\Downloads\2011-01-01~2015-12-31\html\결재",
    r"C:\Users\LEEJUHWAN\Downloads\2016-01-01~2020-12-31\html\결재",
    r"C:\Users\LEEJUHWAN\Downloads\2021-01-01~2025-10-31\html\결재",
]

# DB 연결 정보
DB_CONFIG = {
    'host': 'localhost',
    'port': 3306,
    'database': 'any_approval',
    'user': 'root',
    'password': '1234',
    'charset': 'utf8mb4'
}

print("설정 완료")
print(f"HTML 폴더 수: {len(HTML_FOLDERS)}개")

설정 완료
HTML 폴더 수: 4개


---
## 셀 2: 유틸리티 함수 정의

In [3]:
def get_db_connection():
    """DB 연결 생성"""
    return mysql.connector.connect(**DB_CONFIG)


def extract_doc_id_from_filename(filename):
    """
    파일명에서 문서번호 추출
    예: '20240726_[하계휴가 7_29~30] 휴가신청_23447133.html' -> '23447133'
    """
    # 파일명에서 확장자 제거
    name = os.path.splitext(filename)[0]
    # 마지막 언더스코어 뒤의 숫자 추출
    match = re.search(r'_(\d+)$', name)
    if match:
        return match.group(1)
    return None


def datetime_to_timestamp_ms(dt_str):
    """
    날짜 문자열을 Unix timestamp (밀리초)로 변환
    예: '2024-07-26 13:54:40' -> 1721969680000
    """
    try:
        dt = datetime.strptime(dt_str.strip(), '%Y-%m-%d %H:%M:%S')
        return int(dt.timestamp() * 1000)
    except:
        return 0


def timestamp_ms_to_datetime(ts_ms):
    """
    Unix timestamp (밀리초)를 날짜 문자열로 변환
    예: 1721969680000 -> '2024-07-26 13:54:40'
    """
    try:
        dt = datetime.fromtimestamp(ts_ms / 1000)
        return dt.strftime('%Y-%m-%d %H:%M:%S')
    except:
        return ''


def extract_writer_name(writer_full):
    """
    작성자 전체 문자열에서 이름만 추출
    예: '임현준/팀장/DX사업팀' -> '임현준'
    """
    if not writer_full:
        return ''
    writer_full = writer_full.strip()
    if '/' in writer_full:
        return writer_full.split('/')[0].strip()
    return writer_full


print("유틸리티 함수 정의 완료")

# 테스트
print(f"\n[테스트]")
print(f"문서번호 추출: {extract_doc_id_from_filename('20240726_[하계휴가 7_29~30] 휴가신청_23447133.html')}")
print(f"날짜→타임스탬프: {datetime_to_timestamp_ms('2024-07-26 13:54:40')}")
print(f"타임스탬프→날짜: {timestamp_ms_to_datetime(1721969680000)}")
print(f"작성자 추출: {extract_writer_name('임현준/팀장/DX사업팀')}")

유틸리티 함수 정의 완료

[테스트]
문서번호 추출: 23447133
날짜→타임스탬프: 1721969680000
타임스탬프→날짜: 2024-07-26 13:54:40
작성자 추출: 임현준


---
## 셀 3: HTML 파싱 함수 정의

In [4]:
def get_text_with_linebreaks(element):
    """
    HTML 요소에서 텍스트 추출 (br 태그를 줄바꿈으로 변환)
    <br>, <br/>, <br /> 모두 처리
    """
    if element is None:
        return ''
    
    # 먼저 br 태그를 특수 마커로 변환
    for br in element.find_all('br'):
        br.replace_with('\n')
    
    # 텍스트 추출
    text = element.get_text()
    
    return text.strip()


def parse_comments_from_html(html_path):
    """
    HTML 파일에서 결재댓글 추출
    
    Returns:
        list of dict: [{'writer': '임현준', 'created_at': '2024-07-26 13:54:40', 'message': '...'}, ...]
        None if error
    """
    comments = []
    
    try:
        # 파일 읽기 (인코딩 처리)
        with open(html_path, 'r', encoding='utf-8') as f:
            html_content = f.read()
    except UnicodeDecodeError:
        try:
            with open(html_path, 'r', encoding='cp949') as f:
                html_content = f.read()
        except:
            return None
    except Exception as e:
        return None
    
    # BeautifulSoup으로 파싱 (lxml 파서 사용)
    soup = BeautifulSoup(html_content, 'lxml')
    
    # 결재댓글 테이블 찾기
    comment_table = soup.find('table', {'summary': '결재댓글'})
    if not comment_table:
        return comments  # 빈 리스트 반환 (댓글 테이블 자체가 없음)
    
    # td 안에서 댓글 추출
    td = comment_table.find('td')
    if not td:
        return comments
    
    # span.F_12_black_b (작성자)와 span.F_11_gray (날짜) 쌍을 찾기
    writer_spans = td.find_all('span', class_='F_12_black_b')
    
    for writer_span in writer_spans:
        comment = {}
        
        # 작성자 추출
        writer_full = writer_span.get_text().strip()
        comment['writer'] = extract_writer_name(writer_full)
        
        # 날짜 추출 (작성자 span 다음의 F_11_gray span)
        date_span = writer_span.find_next_sibling('span', class_='F_11_gray')
        if date_span:
            comment['created_at'] = date_span.get_text().strip()
        else:
            # 부모에서 찾기
            parent = writer_span.parent
            if parent:
                date_span = parent.find('span', class_='F_11_gray')
                if date_span:
                    comment['created_at'] = date_span.get_text().strip()
                else:
                    comment['created_at'] = ''
            else:
                comment['created_at'] = ''
        
        # 댓글 내용 추출 (작성자 span 이후의 div 안의 내용)
        # 구조: span.user > br > div > (pre 또는 텍스트)
        message = ''
        
        # writer_span의 부모(span.user)를 찾고, 그 다음 div를 찾기
        parent_span = writer_span.parent
        if parent_span and parent_span.name == 'span':
            # 다음 형제 요소들 중 div 찾기
            next_elem = parent_span.next_sibling
            while next_elem:
                if hasattr(next_elem, 'name') and next_elem.name == 'div':
                    # div 찾음, 그 안의 pre 또는 텍스트 추출
                    pre = next_elem.find('pre')
                    if pre:
                        message = get_text_with_linebreaks(pre)
                    else:
                        message = get_text_with_linebreaks(next_elem)
                    break
                next_elem = next_elem.next_sibling
        
        comment['message'] = message
        
        # 유효한 댓글만 추가 (작성자 또는 내용이 있는 경우)
        if comment['writer'] or comment['message']:
            comments.append(comment)
    
    return comments


print("HTML 파싱 함수 정의 완료")

HTML 파싱 함수 정의 완료


---
## 셀 4: HTML 파일 목록 수집

In [5]:
def collect_html_files():
    """
    모든 폴더에서 HTML 파일 경로 수집
    
    Returns:
        dict: {문서번호: 파일경로}
    """
    html_files = {}
    folder_stats = {}
    
    for folder in HTML_FOLDERS:
        if not os.path.exists(folder):
            print(f"⚠️ 폴더 없음: {folder}")
            continue
        
        folder_name = os.path.basename(os.path.dirname(os.path.dirname(folder)))
        count = 0
        
        # 하위 연도 폴더들 순회
        for year_folder in glob.glob(os.path.join(folder, '*')):
            if os.path.isdir(year_folder):
                for html_file in glob.glob(os.path.join(year_folder, '*.html')):
                    doc_id = extract_doc_id_from_filename(os.path.basename(html_file))
                    if doc_id:
                        html_files[doc_id] = html_file
                        count += 1
        
        folder_stats[folder_name] = count
    
    return html_files, folder_stats


# HTML 파일 수집
print("HTML 파일 수집 중...")
html_files_dict, folder_stats = collect_html_files()

print(f"\n=== HTML 파일 스캔 결과 ===")
print(f"총 파일 수: {len(html_files_dict):,}개\n")

print("폴더별:")
for folder_name, count in folder_stats.items():
    print(f"  - {folder_name}: {count:,}개")

HTML 파일 수집 중...

=== HTML 파일 스캔 결과 ===
총 파일 수: 23,306개

폴더별:
  - 2010-01-01~2010-12-31: 840개
  - 2011-01-01~2015-12-31: 6,587개
  - 2016-01-01~2020-12-31: 7,704개
  - 2021-01-01~2025-10-31: 8,175개


---
## 셀 5: DB 댓글 데이터 로드

In [6]:
# DB에서 댓글 데이터 로드
conn = get_db_connection()
query = """
SELECT 
    id,
    source_id,
    source_document_id,
    created_at,
    updated_at,
    writer,
    message
FROM comments
ORDER BY source_document_id, created_at
"""

db_comments_df = pd.read_sql(query, conn)
conn.close()

# 날짜 컬럼 추가 (읽기 쉽게)
db_comments_df['created_at_str'] = db_comments_df['created_at'].apply(timestamp_ms_to_datetime)

print(f"=== DB comments 테이블 현황 ===")
print(f"총 레코드: {len(db_comments_df)}개")
print(f"문서 수: {db_comments_df['source_document_id'].nunique()}개 (중복 제외)")

# 중복 체크 (source_id 기준)
duplicate_source_ids = db_comments_df[db_comments_df.duplicated(subset=['source_id'], keep=False)]
if len(duplicate_source_ids) > 0:
    print(f"\n⚠️ source_id 중복 레코드 발견: {len(duplicate_source_ids)}건")
    print(duplicate_source_ids[['source_id', 'writer', 'created_at_str', 'message']].head(10))
else:
    print(f"\n✅ source_id 중복 없음")

# 연도별 통계
db_comments_df['year'] = pd.to_datetime(db_comments_df['created_at_str']).dt.year
print(f"\n연도별:")
year_counts = db_comments_df.groupby('year').size()
for year, count in year_counts.items():
    print(f"  - {year}: {count}개")

print(f"\n기간: {db_comments_df['created_at_str'].min()} ~ {db_comments_df['created_at_str'].max()}")

=== DB comments 테이블 현황 ===
총 레코드: 100개
문서 수: 86개 (중복 제외)

✅ source_id 중복 없음

연도별:
  - 2020: 5개
  - 2021: 19개
  - 2022: 14개
  - 2023: 5개
  - 2024: 27개
  - 2025: 30개

기간: 2020-11-09 11:23:05 ~ 2025-12-16 13:36:17


In [7]:
# DB 댓글 샘플 확인
print("=== DB 댓글 샘플 (처음 5개) ===")
db_comments_df[['source_document_id', 'writer', 'created_at_str', 'message']].head()

=== DB 댓글 샘플 (처음 5개) ===


,source_document_id,writer,created_at_str,message
0,14835264,최규,2020-11-09 11:23:05,승인합니다.
1,14884099,박상택,2020-11-17 15:52:40,최종 결재자는 승인이 되어야 합니다
2,14967986,최기원,2020-12-02 10:15:48,원본 영수증도 보내주세요
3,15083828,송준선,2020-12-21 09:37:28,- ETRI 통합정보시스템 유지관리 관련 11월 비용 지급을 합의합니다.
4,15119275,조윤호,2020-12-24 10:34:13,감사합니다.


---
# 🔍 1단계: DB 댓글 기준 HTML 파싱 검증

DB에 있는 100개 댓글을 HTML에서 파싱한 결과와 비교하여 파싱 로직이 정확한지 검증합니다.

In [8]:
# DB에 있는 댓글의 문서번호 목록
db_doc_ids = db_comments_df['source_document_id'].unique().tolist()
print(f"DB에 있는 문서번호: {len(db_doc_ids)}개")

# HTML 파일이 있는 문서번호 필터링
db_doc_ids_with_html = [doc_id for doc_id in db_doc_ids if doc_id in html_files_dict]
db_doc_ids_without_html = [doc_id for doc_id in db_doc_ids if doc_id not in html_files_dict]

print(f"HTML 파일 있는 문서: {len(db_doc_ids_with_html)}개")
print(f"HTML 파일 없는 문서: {len(db_doc_ids_without_html)}개")

if db_doc_ids_without_html:
    print(f"\n📋 HTML 파일 없는 문서 목록 (DB에 있지만 HTML 없음):")
    for doc_id in db_doc_ids_without_html:
        doc_comments = db_comments_df[db_comments_df['source_document_id'] == doc_id]
        for _, row in doc_comments.iterrows():
            print(f"  - 문서번호: {doc_id}, 작성일: {row['created_at_str']}, 작성자: {row['writer']}")

DB에 있는 문서번호: 86개
HTML 파일 있는 문서: 85개
HTML 파일 없는 문서: 1개

📋 HTML 파일 없는 문서 목록 (DB에 있지만 HTML 없음):
  - 문서번호: 26707524, 작성일: 2025-12-16 13:36:17, 작성자: 이용일


In [9]:
# HTML에서 댓글 파싱 및 DB와 비교
print("=== HTML 파싱 검증 시작 ===")
print(f"검증 대상: {len(db_doc_ids_with_html)}개 문서\n")

validation_results = []
parse_errors = []

for doc_id in tqdm(db_doc_ids_with_html, desc="HTML 파싱 중"):
    html_path = html_files_dict[doc_id]
    
    # HTML에서 댓글 파싱
    html_comments = parse_comments_from_html(html_path)
    
    if html_comments is None:
        parse_errors.append({'doc_id': doc_id, 'error': '파일 읽기 실패', 'path': html_path})
        continue
    
    # DB에서 해당 문서의 댓글 가져오기
    db_doc_comments = db_comments_df[db_comments_df['source_document_id'] == doc_id].copy()
    
    # 비교
    for idx, db_row in db_doc_comments.iterrows():
        result = {
            'doc_id': doc_id,
            'db_writer': db_row['writer'],
            'db_created_at': db_row['created_at_str'],
            'db_message': db_row['message'],
            'html_writer': None,
            'html_created_at': None,
            'html_message': None,
            'match_status': 'NOT_FOUND',
            'detail': ''
        }
        
        # HTML 댓글에서 매칭되는 것 찾기 (작성자 + 날짜 기준)
        for html_comment in html_comments:
            if (html_comment['writer'] == db_row['writer'] and 
                html_comment['created_at'] == db_row['created_at_str']):
                
                result['html_writer'] = html_comment['writer']
                result['html_created_at'] = html_comment['created_at']
                result['html_message'] = html_comment['message']
                
                # 내용 비교
                db_msg = (db_row['message'] or '').strip()
                html_msg = (html_comment['message'] or '').strip()
                
                if db_msg == html_msg:
                    result['match_status'] = 'EXACT_MATCH'
                elif db_msg.replace('\r\n', '\n') == html_msg.replace('\r\n', '\n'):
                    result['match_status'] = 'MATCH_CRLF_DIFF'
                    result['detail'] = '줄바꿈 문자 차이 (\\r\\n vs \\n)'
                elif db_msg.replace(' ', '').replace('\n', '') == html_msg.replace(' ', '').replace('\n', ''):
                    result['match_status'] = 'MATCH_WHITESPACE_DIFF'
                    result['detail'] = '공백/줄바꿈 차이'
                else:
                    result['match_status'] = 'CONTENT_MISMATCH'
                    result['detail'] = f'DB길이:{len(db_msg)}, HTML길이:{len(html_msg)}'
                break
        
        validation_results.append(result)

validation_df = pd.DataFrame(validation_results)
print(f"\n검증 완료: {len(validation_df)}개 댓글")

=== HTML 파싱 검증 시작 ===
검증 대상: 85개 문서



HTML 파싱 중: 100%|████████████████████████████████████████████████████████████████████| 85/85 [00:03<00:00, 27.07it/s]


검증 완료: 99개 댓글


In [10]:
# 검증 결과 요약
print("=== 파싱 검증 결과 요약 ===")

status_counts = validation_df['match_status'].value_counts()
total = len(validation_df)

for status, count in status_counts.items():
    pct = count / total * 100
    if status == 'EXACT_MATCH':
        print(f"  ✅ 완전 일치: {count}개 ({pct:.1f}%)")
    elif status == 'MATCH_CRLF_DIFF':
        print(f"  ⚠️ 줄바꿈 문자 차이만 있음: {count}개 ({pct:.1f}%)")
    elif status == 'MATCH_WHITESPACE_DIFF':
        print(f"  ⚠️ 공백/줄바꿈 차이만 있음: {count}개 ({pct:.1f}%)")
    elif status == 'CONTENT_MISMATCH':
        print(f"  ❌ 내용 불일치: {count}개 ({pct:.1f}%)")
    elif status == 'NOT_FOUND':
        print(f"  ❓ HTML에서 못 찾음: {count}개 ({pct:.1f}%)")

if parse_errors:
    print(f"\n⚠️ 파싱 오류 발생: {len(parse_errors)}건")
    for err in parse_errors:
        print(f"  - 문서번호: {err['doc_id']}, 오류: {err['error']}")

# 일치율 계산
match_count = len(validation_df[validation_df['match_status'].isin(['EXACT_MATCH', 'MATCH_CRLF_DIFF', 'MATCH_WHITESPACE_DIFF'])])
match_rate = match_count / total * 100 if total > 0 else 0
print(f"\n📊 전체 일치율: {match_rate:.1f}% ({match_count}/{total})")

=== 파싱 검증 결과 요약 ===
  ✅ 완전 일치: 99개 (100.0%)

📊 전체 일치율: 100.0% (99/99)


In [11]:
# 불일치/미발견 케이스 상세 확인
problem_cases = validation_df[~validation_df['match_status'].isin(['EXACT_MATCH', 'MATCH_CRLF_DIFF'])]

if len(problem_cases) > 0:
    print(f"=== 불일치/미발견 케이스 상세 ({len(problem_cases)}건) ===")
    
    for idx, row in problem_cases.iterrows():
        print(f"\n{'='*60}")
        print(f"[문서번호: {row['doc_id']}] 상태: {row['match_status']}")
        print(f"상세: {row['detail']}")
        print(f"\nDB 댓글:")
        print(f"  작성자: {row['db_writer']}")
        print(f"  날짜: {row['db_created_at']}")
        print(f"  내용: {repr(row['db_message'][:200] if row['db_message'] else '')}...")
        print(f"\nHTML 댓글:")
        print(f"  작성자: {row['html_writer']}")
        print(f"  날짜: {row['html_created_at']}")
        print(f"  내용: {repr(row['html_message'][:200] if row['html_message'] else '')}...")
else:
    print("✅ 모든 댓글이 정상적으로 매칭되었습니다!")

✅ 모든 댓글이 정상적으로 매칭되었습니다!


In [12]:
# 1단계 검증 결론
exact_match_count = len(validation_df[validation_df['match_status'] == 'EXACT_MATCH'])
acceptable_match_count = len(validation_df[validation_df['match_status'].isin(['EXACT_MATCH', 'MATCH_CRLF_DIFF'])])

print("="*60)
print("1단계 검증 결론")
print("="*60)

if exact_match_count == total:
    print(f"\n🎉 완벽! 모든 댓글({total}개)이 100% 완전 일치합니다.")
    print("\n→ 2단계 (전체 HTML 스캔)를 진행해도 됩니다.")
    VALIDATION_PASSED = True
elif acceptable_match_count == total:
    print(f"\n✅ 모든 댓글({total}개)이 일치합니다. (일부 줄바꿈 문자 차이 있음)")
    print("\n→ 2단계 (전체 HTML 스캔)를 진행해도 됩니다.")
    VALIDATION_PASSED = True
else:
    problem_count = total - acceptable_match_count
    print(f"\n⚠️ {problem_count}개 댓글에서 문제가 발견되었습니다.")
    print("\n→ 위의 불일치 케이스를 확인하고 파싱 로직을 수정해야 합니다.")
    VALIDATION_PASSED = False

print(f"\nVALIDATION_PASSED = {VALIDATION_PASSED}")

1단계 검증 결론

🎉 완벽! 모든 댓글(99개)이 100% 완전 일치합니다.

→ 2단계 (전체 HTML 스캔)를 진행해도 됩니다.

VALIDATION_PASSED = True


---
# 📂 2단계: 전체 HTML 스캔 및 누락 댓글 추출

⚠️ **1단계 검증이 통과한 경우에만 실행하세요!**

In [13]:
# 검증 통과 여부 확인
if not VALIDATION_PASSED:
    print("❌ 1단계 검증이 통과하지 않았습니다.")
    print("파싱 로직을 수정한 후 다시 시도하세요.")
else:
    print("✅ 1단계 검증 통과! 2단계를 진행합니다.")

✅ 1단계 검증 통과! 2단계를 진행합니다.


In [14]:
# 전체 HTML 파일에서 댓글 추출
if VALIDATION_PASSED:
    print("=== 전체 HTML 파일 스캔 시작 ===")
    print(f"스캔 대상: {len(html_files_dict):,}개 파일\n")
    
    all_html_comments = {}  # {doc_id: [comments]}
    files_with_comments = 0
    files_without_comments = 0
    scan_errors = []
    year_stats = defaultdict(lambda: {'total': 0, 'with_comments': 0, 'comment_count': 0})
    
    for doc_id, html_path in tqdm(html_files_dict.items(), desc="HTML 스캔 중"):
        # 연도 추출 (파일 경로에서)
        year_match = re.search(r'\\(\d{4})\\', html_path)
        year = year_match.group(1) if year_match else 'unknown'
        year_stats[year]['total'] += 1
        
        # HTML 파싱
        comments = parse_comments_from_html(html_path)
        
        if comments is None:
            scan_errors.append({'doc_id': doc_id, 'path': html_path})
            continue
        
        if comments:
            all_html_comments[doc_id] = comments
            files_with_comments += 1
            year_stats[year]['with_comments'] += 1
            year_stats[year]['comment_count'] += len(comments)
        else:
            files_without_comments += 1
    
    total_comments = sum(len(c) for c in all_html_comments.values())
    
    print(f"\n=== HTML 스캔 결과 ===")
    print(f"총 HTML 파일: {len(html_files_dict):,}개")
    print(f"├─ 댓글 있는 파일: {files_with_comments:,}개 ({files_with_comments/len(html_files_dict)*100:.1f}%)")
    print(f"└─ 댓글 없는 파일: {files_without_comments:,}개 ({files_without_comments/len(html_files_dict)*100:.1f}%)")
    print(f"\n총 추출된 댓글 수: {total_comments:,}개")
    
    if scan_errors:
        print(f"\n⚠️ 스캔 오류: {len(scan_errors)}건")

=== 전체 HTML 파일 스캔 시작 ===
스캔 대상: 23,306개 파일



HTML 스캔 중: 100%|██████████████████████████████████████████████████████████████| 23306/23306 [11:53<00:00, 32.67it/s]


=== HTML 스캔 결과 ===
총 HTML 파일: 23,306개
├─ 댓글 있는 파일: 85개 (0.4%)
└─ 댓글 없는 파일: 23,221개 (99.6%)

총 추출된 댓글 수: 99개


In [15]:
# 연도별 댓글 현황
if VALIDATION_PASSED:
    print("=== 연도별 댓글 현황 ===")
    print(f"{'연도':<6} {'총파일':>10} {'댓글있음':>10} {'비율':>8} {'댓글수':>10}")
    print("-" * 50)
    
    for year in sorted(year_stats.keys()):
        stats = year_stats[year]
        rate = stats['with_comments'] / stats['total'] * 100 if stats['total'] > 0 else 0
        print(f"{year:<6} {stats['total']:>10,} {stats['with_comments']:>10,} {rate:>7.1f}% {stats['comment_count']:>10,}")

=== 연도별 댓글 현황 ===
연도            총파일       댓글있음       비율        댓글수
--------------------------------------------------
2010          840          0     0.0%          0
2011          855          0     0.0%          0
2012        1,280          0     0.0%          0
2013        1,585          0     0.0%          0
2014        1,502          0     0.0%          0
2015        1,365          0     0.0%          0
2016        1,527          0     0.0%          0
2017        1,963          0     0.0%          0
2018        1,661          0     0.0%          0
2019        1,478          0     0.0%          0
2020        1,075          5     0.5%          5
2021        1,331         19     1.4%         19
2022        1,818         13     0.7%         14
2023        1,442          6     0.4%          6
2024        1,811         22     1.2%         26
2025        1,773         20     1.1%         29


In [18]:
# 2010~2019년 HTML 중 결재댓글 td에 내용이 있는 파일 찾기
import os
from bs4 import BeautifulSoup

check_years = ['2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019']
found_files = []

for doc_id, html_path in html_files_dict.items():
    # 2010~2019년 파일만
    year_match = re.search(r'\\(\d{4})\\', html_path)
    if not year_match or year_match.group(1) not in check_years:
        continue
    
    try:
        with open(html_path, 'r', encoding='utf-8') as f:
            soup = BeautifulSoup(f.read(), 'lxml')
        
        comment_table = soup.find('table', {'summary': '결재댓글'})
        if comment_table:
            td = comment_table.find('td')
            if td:
                # td 안의 텍스트 길이 확인 (공백 제거)
                text = td.get_text(strip=True)
                if len(text) > 10:  # 의미있는 내용이 있으면
                    found_files.append({
                        'doc_id': doc_id,
                        'year': year_match.group(1),
                        'path': html_path,
                        'text_preview': text[:100]
                    })
                    if len(found_files) >= 5:  # 5개만 찾으면 중단
                        break
    except:
        continue

print(f"2010~2019년 중 결재댓글 내용이 있는 파일: {len(found_files)}개")
for f in found_files:
    print(f"\n연도: {f['year']}, 문서번호: {f['doc_id']}")
    print(f"내용 미리보기: {f['text_preview']}")

2010~2019년 중 결재댓글 내용이 있는 파일: 0개


In [16]:
# DB와 비교하여 누락 댓글 찾기
if VALIDATION_PASSED:
    print("=== DB와 HTML 비교 ===")
    
    # HTML에서 추출한 문서번호
    html_doc_ids_with_comments = set(all_html_comments.keys())
    
    # DB에 있는 문서번호
    db_doc_ids_set = set(db_comments_df['source_document_id'].unique())
    
    # 비교
    both = html_doc_ids_with_comments & db_doc_ids_set
    html_only = html_doc_ids_with_comments - db_doc_ids_set
    db_only = db_doc_ids_set - html_doc_ids_with_comments
    
    print(f"\n[문서 단위 비교]")
    print(f"HTML에서 댓글 있는 문서: {len(html_doc_ids_with_comments):,}개")
    print(f"DB에 있는 문서: {len(db_doc_ids_set)}개")
    print(f"├─ 교집합 (둘 다 있음): {len(both)}개")
    print(f"├─ HTML에만 있음 (DB 누락): {len(html_only):,}개  ← INSERT 대상")
    print(f"└─ DB에만 있음 (HTML에 없음): {len(db_only)}개")

=== DB와 HTML 비교 ===

[문서 단위 비교]
HTML에서 댓글 있는 문서: 85개
DB에 있는 문서: 86개
├─ 교집합 (둘 다 있음): 85개
├─ HTML에만 있음 (DB 누락): 0개  ← INSERT 대상
└─ DB에만 있음 (HTML에 없음): 1개


In [17]:
# DB에만 있는 댓글 상세 (확인용)
if VALIDATION_PASSED and db_only:
    print("=== DB에 있지만 HTML에 없는 댓글 (확인용) ===")
    print(f"총 {len(db_only)}개 문서\n")
    
    db_only_comments = db_comments_df[db_comments_df['source_document_id'].isin(db_only)]
    print(f"{'문서번호':<12} {'작성일':<20} {'작성자':<10}")
    print("-" * 45)
    
    for _, row in db_only_comments.iterrows():
        print(f"{row['source_document_id']:<12} {row['created_at_str']:<20} {row['writer']:<10}")

=== DB에 있지만 HTML에 없는 댓글 (확인용) ===
총 1개 문서

문서번호         작성일                  작성자       
---------------------------------------------
26707524     2025-12-16 13:36:17  이용일       


In [ ]:
# 누락 댓글 데이터 준비 (INSERT 대상)
if VALIDATION_PASSED:
    print("=== INSERT 대상 데이터 준비 ===")
    
    missing_comments = []
    
    for doc_id in html_only:
        comments = all_html_comments[doc_id]
        
        for idx, comment in enumerate(comments, 1):
            # source_id 생성: 순번_문서번호_01
            source_id = f"{idx:02d}_{doc_id}_01"
            
            # 타임스탬프 변환
            created_at_ms = datetime_to_timestamp_ms(comment['created_at'])
            
            missing_comments.append({
                'source_id': source_id,
                'source_document_id': doc_id,
                'created_at': created_at_ms,
                'updated_at': created_at_ms,
                'writer': comment['writer'],
                'message': comment['message'],
                'created_at_str': comment['created_at']  # 확인용
            })
    
    missing_df = pd.DataFrame(missing_comments)
    
    print(f"총 {len(html_only):,}개 문서에서 {len(missing_df):,}개 댓글")
    
    # 연도별 누락 현황
    if len(missing_df) > 0:
        missing_df['year'] = pd.to_datetime(missing_df['created_at_str'], errors='coerce').dt.year
        print(f"\n연도별 누락 현황:")
        year_missing = missing_df.groupby('year').size()
        for year, count in year_missing.items():
            print(f"  - {int(year) if pd.notna(year) else 'N/A'}: {count}개")

In [ ]:
# 누락 댓글 샘플 확인
if VALIDATION_PASSED and len(missing_df) > 0:
    print("=== 누락 댓글 샘플 (처음 10개) ===")
    display(missing_df[['source_id', 'source_document_id', 'created_at_str', 'writer', 'message']].head(10))

In [ ]:
# CSV 파일로 저장 (백업용)
if VALIDATION_PASSED and len(missing_df) > 0:
    csv_path = 'missing_comments.csv'
    missing_df.to_csv(csv_path, index=False, encoding='utf-8-sig')
    print(f"✅ CSV 저장 완료: {csv_path}")
    print(f"   총 {len(missing_df):,}개 레코드")

---
# 💾 3단계: DB INSERT

⚠️ **신중하게 실행하세요! 실행 전 위의 데이터를 반드시 확인하세요.**

In [ ]:
# INSERT 전 최종 확인
if VALIDATION_PASSED and len(missing_df) > 0:
    print("=== INSERT 전 최종 확인 ===")
    print(f"INSERT 대상: {len(missing_df):,}개 댓글")
    print(f"\n⚠️ 아래 셀을 실행하면 DB에 데이터가 추가됩니다.")
    print(f"   실행 전 위의 샘플 데이터를 반드시 확인하세요.")
else:
    print("INSERT할 데이터가 없습니다.")

In [ ]:
# DB INSERT 실행
# ⚠️ 주의: 이 셀은 수동으로 실행하세요!

DO_INSERT = False  # True로 변경하면 실행됨

if DO_INSERT and VALIDATION_PASSED and len(missing_df) > 0:
    print("=== DB INSERT 시작 ===")
    
    conn = get_db_connection()
    cursor = conn.cursor()
    
    insert_query = """
    INSERT INTO comments (source_id, source_document_id, created_at, updated_at, writer, message)
    VALUES (%s, %s, %s, %s, %s, %s)
    """
    
    success_count = 0
    skip_count = 0
    error_count = 0
    errors = []
    
    for _, row in tqdm(missing_df.iterrows(), total=len(missing_df), desc="INSERT 중"):
        try:
            cursor.execute(insert_query, (
                row['source_id'],
                row['source_document_id'],
                row['created_at'],
                row['updated_at'],
                row['writer'],
                row['message']
            ))
            success_count += 1
        except mysql.connector.IntegrityError as e:
            # source_id 중복 (이미 존재)
            skip_count += 1
        except Exception as e:
            error_count += 1
            errors.append({'source_id': row['source_id'], 'error': str(e)})
    
    conn.commit()
    cursor.close()
    conn.close()
    
    print(f"\n=== INSERT 결과 ===")
    print(f"✅ 성공: {success_count:,}개")
    print(f"⏭️ 중복 스킵: {skip_count}개")
    print(f"❌ 실패: {error_count}개")
    
    if errors:
        print(f"\n오류 상세:")
        for err in errors[:10]:
            print(f"  - {err['source_id']}: {err['error']}")
else:
    if not DO_INSERT:
        print("DO_INSERT = False 입니다.")
        print("INSERT를 실행하려면 위에서 DO_INSERT = True 로 변경하세요.")

In [ ]:
# INSERT 후 확인
if DO_INSERT and VALIDATION_PASSED:
    conn = get_db_connection()
    query = "SELECT COUNT(*) as cnt FROM comments"
    result = pd.read_sql(query, conn)
    conn.close()
    
    print(f"=== INSERT 후 DB 현황 ===")
    print(f"comments 테이블 총 레코드: {result['cnt'].values[0]:,}개")